<img src='pic/logo.png'/>


   # DJANGO-REST-FRAMEWORK TUTORIAL
   
   ## Majid Iranpour
   ## Twitter: @_majidmc2
<br><br><br>
   <hr>
   <br><br><br>
   

# @api_view and Models

## Example (Employess):
 

###  models.py:

In [ ]:
from django.db import models


class Employ(models.Model):
    name = models.CharField(max_length=40)
    age = models.IntegerField()
    salary = models.IntegerField()
    post = models.CharField(max_length=30)
    created_at = models.DateTimeField(null=True, blank=True)
    updated_at = models.DateTimeField(null=True, blank=True)

    def __str__(self):
        return self.name


### serializers.py

In [ ]:
from rest_framework import serializers
from .models import Employ


class EmploySerializer(serializers.ModelSerializer):
    class Meta:
        model = Employ
        fields = '__all__'  # fields = ('id', 'name', 'age', 'salary', 'post', 'created_at', 'updated_at')

    def validate_name(self, value):
        if value == "Ahmad":
            raise serializers.ValidationError("He's blocked!!!")
        return value

    #     def save(self):
    #         name = self.validated_data['name']
    #         print("SAVE {}".format(name))

    def create(self, validated_data):
        instance = super().create(validated_data)

        instance.created_at = timezone.now()
        instance.save()
        return instance

    def update(self, instance, validated_data):
        old_created_at = instance.created_at

        new_instance = super().create(validated_data, validated_data)

        new_instance.created_at = old_created_at
        new_instance.updated_at = timezone.now()
        new_instance.save()
        return new_instance
  

### views.py

#### POST Request

In [ ]:
from .models import Employ
from .serializers import EmploySerializer

from rest_framework import status


@api_view(['POST'])
def post_employ(request):

    data = {
        'name': request.data['name'],
        'age': request.data['age'],
        'salary': request.data['salary'],
        'post': request.data['post']
    }
    serializer = EmploySerializer(data=data)
    if serializer.is_valid():
        serializer.save()
        return Response(serializer.data, status=status.HTTP_201_CREATED)
    return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


#### GET All Request

In [1]:
@api_view(['GET'])
def get_all_employees(request):
    employees = Employ.objects.all()
    serializer = EmploySerializer(employees, many=True)
    return Response(serializer.data, status=status.HTTP_200_OK)

NameError: name 'api_view' is not defined

#### GET Single and UPDATE and DELETE Request

In [ ]:
@api_view(['GET', 'PUT', 'DELETE'])
def get_delete_update_employ(request, pk):
    
    try:
        employ = Employ.objects.get(pk=pk)
    except Employ.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        serializer = EmploySerializer(employ)
        return Response(serializer.data, status=status.HTTP_200_OK)

    if request.method == 'PUT':
        serializer = EmploySerializer(employ, data=request.data)
        if serializer.is_valid():
            serializer.save()
            return Response(serializer.data, status=status.HTTP_200_OK)
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    if request.method == 'DELETE':
        employ.delete()
        return Response(status=status.HTTP_204_NO_CONTENT)

#### Search


In [ ]:
@api_view(['GET'])
def search_employ(request):

    try:
        employees = Employ.objects.all().filter(name=request.query_params['name'])
    except Employ.DoesNotExist:
        return Response(status=status.HTTP_404_NOT_FOUND)

    serializer = EmploySerializer(employees, many=True)
    return Response(serializer.data, status=status.HTTP_200_OK)


### urls.py

In [ ]:
urlpatterns = [
    .
    .
    .
    path(r'post_employ', views.post_employ, name='post_employ'),
    path(r'get_all_employees', views.get_all_employees, name='get_all_employees'),
    path(r'get_delete_update_employ/<int:pk>', views.get_delete_update_employ, name='get_delete_update_employ'),
    path(r'search_employ', views.search_employ, name='search_employ'),
    .
    .
    .
]


# Serializer
## Example(Serveres):
### models.py:

In [ ]:
    
class Server(models.Model):
    employ = models.ForeignKey(Employ, on_delete=models.CASCADE)
    server_name = models.CharField(max_length=40)
    
    def __str__(self):
        return self.server_name


### serializers.py:

In [ ]:
    
class WriteServerSerializer(serializers.ModelSerializer):
    class Meta:
        model = Server
        fields = '__all__'


class ReadServerSerializer(serializers.ModelSerializer):
    class Meta:
        model = Server
        fields = '__all__'
        depth = 1


### views.py



In [ ]:

@api_view(['POST'])
def post_server_and_employ(request):
    employ_data = {
        'name': request.data['employ']['name'],
        'age': request.data['employ']['age'],
        'salary': request.data['employ']['salary'],
        'post': request.data['employ']['post'],
    }

    employ_serializer = EmploySerializer(data=employ_data)

    if employ_serializer.is_valid():
        employ_serializer.save()
    else:
        return Response(employ_serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    server_data = {
        'server_name': request.data['server_name'],
        'employ': employ_serializer.data['id']
    }

    server_serializer = WriteServerSerializer(data=server_data)

    if server_serializer.is_valid():
        server_serializer.save()
        return Response(server_serializer.data, status=status.HTTP_201_CREATED)

    return Response(server_serializer.errors, status=status.HTTP_400_BAD_REQUEST)


"""
{
    "server_name":"dhcp",
    "employ":{
        "name":"Hassan",
        "age":30,
        "salary":5000,
        "post":"DEV"
    }
}
"""


@api_view(['POST'])
def post_server(request):
    try:
        employ = Employ.objects.filter(name=request.data['employ'])[0]
    except Employ.DoesNotExist:
        return Response({"error": "user not found!"}, status=status.HTTP_404_NOT_FOUND)

    server_data = {
        'server_name': request.data['server_name'],
        'employ': employ.id
    }

    serializer = WriteServerSerializer(data=server_data)

    if serializer.is_valid():
        serializer.save()
        return Response(serializer.data, status=status.HTTP_201_CREATED)

    return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


"""
{
    "server_name":"dnc",
    "employ":"Hassan"
}
"""


@api_view(['GET'])
def get_all_servers(request):
    employees = Server.objects.all()
    serializer = ReadServerSerializer(employees, many=True)
    return Response(serializer.data, status=status.HTTP_200_OK)


### urls.py

In [ ]:
urlpatterns = [
    .
    .
    .
    path(r'post_server_and_employ', views.post_server_and_employ, name='post_server_and_employ'),
    path(r'post_server', views.post_server, name='post_server'),
    path(r'get_all_servers', views.get_all_servers, name='get_all_servers'),
    .
    .
    .
]


# ModelViewSet

## Example(Book):

### models.py

In [ ]:
from django.db import models


class Book(models.Model):
    name = models.CharField(max_length=40)
    pages = models.IntegerField()
    auther = models.CharField(max_length=30)
    type = models.CharField(max_length=30)
    created_at = models.DateTimeField(auto_now_add=True)

    def __str__(self):
        return self.name

### serializers.py:

In [ ]:
from rest_framework import serializers
from .models import Book


class BookSerializer(serializers.ModelSerializer):
    class Meta:
        model = Book
        fields = '__all__'

### views.py

In [ ]:
from rest_framework import viewsets

from .serializers import BookSerializer
from .models import Book


# class BookViewSet(viewsets.ModelViewSet):
#     queryset = Book.objects.all()
#     serializer_class = BookSerializer
#     http_method_names = ['get', 'post', 'put', 'patch', 'delete']
    

class BookViewSet(viewsets.ModelViewSet):
    queryset = Book.objects.all()
    serializer_class = BookSerializer
    http_method_names = ['get', 'post', 'put', 'patch', 'delete']
    
    def list(self, request, *args, **kwargs):
        response = super().list(request, *kwargs, **kwargs)
        print("====== list ======")
        return response

    def create(self, request, *args, **kwargs):
        response = super().create(request, *kwargs, **kwargs)
        instance = self.get_object()
        print("====== create ======")
        return response

    def update(self, request, *args, **kwargs):
        response = super().update(request, *kwargs, **kwargs)
        instance = self.get_object()
        print("====== update : {} ======".format(instance.name))
        return response
    
    def retrieve(self, request, *args, **kwargs):
        response = super().retrieve(request, *kwargs, **kwargs)
        instance = self.get_object()
        print("====== retrieve : {} ======".format(instance.name))
        return response

    def destroy(self, request, *args, **kwargs):
        response = super().destroy(request, *kwargs, **kwargs)
        instance = self.get_object()
        print("====== destroy : {} ======".format(instance.name))
        return response


### urls.py

In [ ]:
from django.urls import path
from myApp import views
from rest_framework.routers import DefaultRouter

router = DefaultRouter()
router.register(r'book', views.BookViewSet, basename='BookViewset')

urlpatterns = [
    .
    .
    .
]

urlpatterns += router.urls